# Uses the PiNN model to generate a repulsive potential

Adds in the calcualtion to 

E_tot = E_DFTB + E_CCS + E_PiNN + ...

In [2]:
import os, warnings
import tensorflow as tf
from glob import glob
from collections import OrderedDict, defaultdict
from ase.collections import g2
import pinn
from pinn.io import load_ase, sparse_batch
#from pinn.models import potential_model
#from pinn.calculator import PiNN_calc
import warnings
warnings.filterwarnings('ignore')
# CPU is used for documentation generation, feel free to use your GPU!
os.environ['CUDA_VISIBLE_DEVICES'] = ''
import ase.db
base_dir=os.getcwd()
os.chdir(base_dir)
print('You are here:', base_dir)

# We heavily use indexed slices to do sparse summations,
# which causes tensorflow to complain,
# we believe it's safe to ignore this warning.
index_warning = 'Converting sparse IndexedSlices'
warnings.filterwarnings('ignore', index_warning)

You are here: /home/broqvist/Desktop/parautomatik/Electrolyte


In [11]:
def generate_data(DFT_DB, DFTB_DB, CCS_DB): 
    cwd = os.getcwd()
    if not os.path.isdir(cwd+"/PiNN/"):
        os.mkdir(cwd+"/PiNN/")

    os.chdir(cwd+"/PiNN/")
    f=open('trainset.xyz', 'a')
    r=open('erep.dat', 'a')
    fu=open('trainset_udftb.xyz', 'a')
    ru=open('erep_udftb.dat', 'a')

    for row in DFTB_DB.select():
        structure = row.toatoms()
        id=str(row.key)
        EDFTB=row.energy
        FDFTB=row.forces
        EDFT=DFT_DB.get('key='+id).energy
        FDFT=DFT_DB.get('key='+id).forces
        ECCS=CCS_DB.get('key='+id).energy
        FCCS=CCS_DB.get('key='+id).forces
        Erep=EDFT-EDFTB-ECCS
        Frep=FDFT-FDFTB-FCCS
        r.write(str(Erep)+'\n')       
        ru.write(str(EDFT)+'\n')
        lattice=structure.get_cell()
        f.write(''+str(structure.get_global_number_of_atoms())+'\n')
        f.write('Lattice="'+str(lattice[0,0])+' '+str(lattice[0,1])+' '+str(lattice[0,2])+' '+str(lattice[1,0])+' '+str(lattice[1,1])+' '+str(lattice[1,2])+' '+str(lattice[2,0])+' '+str(lattice[2,1])+' '+str(lattice[2,2])+'" Properties=species:S:1:pos:R:3:forces:R:3 energy='+str(Erep)+' stress="0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0" pbc="T T T"\n')        
        fu.write(''+str(structure.get_global_number_of_atoms())+'\n')	
        fu.write('Lattice="'+str(lattice[0,0])+' '+str(lattice[0,1])+' '+str(lattice[0,2])+' '+str(lattice[1,0])+' '+str(lattice[1,1])+' '+str(lattice[1,2])+' '+str(lattice[2,0])+' '+str(lattice[2,1])+' '+str(lattice[2,2])+'" Properties=species:S:1:pos:R:3:forces:R:3 energy='+str(EDFT)+' stress="0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0" pbc="T T T"\n')
        for i in range(structure.get_global_number_of_atoms()):
            position=structure.positions[i]
            fpos=Frep[i]
            f.write(''+str(structure.get_chemical_symbols()[i])+'   '+str(position[0])+' '+str(position[1])+' '+str(position[2])+'    '+str(fpos[0])+' '+str(fpos[1])+' '+str(fpos[2])+'\n')
            position=structure.positions[i]
            fpos=FDFT[i]
            fu.write(''+str(structure.get_chemical_symbols()[i])+'   '+str(position[0])+' '+str(position[1])+' '+str(position[2])+'    '+str(fpos[0])+' '+str(fpos[1])+' '+str(fpos[2])+'\n')
            

            

def generate_data(DFT_DB, DFTB_DB, CCS_DB): 
    cwd = os.getcwd()
    if not os.path.isdir(cwd+"/PiNN/"):
        os.mkdir(cwd+"/PiNN/")

    os.chdir(cwd+"/PiNN/")
    dataset=OrderedDict()
    print(len(DFTB_DB))
    for row in DFTB_DB.select():
        structure = row.toatoms()
        id=str(row.key)
        EDFTB=row.energy
        FDFTB=row.forces
        EDFT=DFT_DB.get('key='+id).energy
        FDFT=DFT_DB.get('key='+id).forces
        ECCS=CCS_DB.get('key='+id).energy
        FCCS=CCS_DB.get('key='+id).forces
        Erep=EDFT-EDFTB-ECCS
        Frep=FDFT-FDFTB-FCCS
        dataset[id] = load_data(structure,Erep, Frep)     
    return dataset

keys = ...
def load_key(key, DB1, DB2, DB3):
     return


def load_data(atoms, Erep, Frep): 
    data = {'elems': atoms.numbers,
            'coord': atoms.positions,
            'cell': atoms.cell,
            'f_data': Frep,
            'e_data': Erep}
    return data

# Generate the dataset 
Here we use the newly generated ase databases (DFTB_DB and CCS_DB) together with the original DFT_DB to create the training set for the training of the machine learning potential. The outpout of this step is 

- trainset.xyz
- erep.dat
- trainset_urep.xyz
- erep_udftb.dat

Here we use a 80:20 test set. 

In [3]:
# generate the data set 
os.chdir(base_dir)
!pwd

db_ccs=ase.db.connect("CCS_DB.db")     # CCS database
db_dft=ase.db.connect("DFT_DB.db")     # DFT database, original keys for structure
db_dftb=ase.db.connect("DFTB_DB.db")   # DFTB database  

generate_data(db_dft,db_dftb,db_ccs)


/home/broqvist/Desktop/parautomatik/Electrolyte


NameError: name 'generate_data' is not defined

In [34]:
os.chdir(base_dir)
cwd = os.getcwd()
os.chdir(cwd+"/PiNN/")

#trainset.xyz -o 'train:9,test:1'     


from pinn.io import load_ase, write_tfrecord


#ds=load_ase('trainset.xyz',splits={'train':8, 'test':2})
#next(iter(ds['train']))

filelist = 'trainset.xyz'
dataset = lambda: load_ase(filelist, splits={'train':8, 'test':2})
train = lambda: dataset()['train'].repeat().shuffle(1000).apply(sparse_batch(1))
test = lambda: dataset()['test'].apply(sparse_batch(1))

In [27]:
!pwd

/home/broqvist/Desktop/parautomatik/Electrolyte/PiNN


In [35]:
from pinn import get_model
params={'model_dir': './',
          'network': {
              'name': 'PiNet',
              'params': {
                  'depth': 4,
                  'rc':4.0,
                  'atom_types':[1,6,7,8,9]
              },
          },
          'model': {
              'name': 'potential_model',
              'params': {
                  'learning_rate': 1e-3
              }},
        'optimizer': {
    'class_name': 'EKF',
    'config': {
        'learning_rate': 0.03}}
}

from pinn.utils import init_params 

init_params(params, ds['train'])

model = get_model(params)

# pinn train -t train.yml -e test.yml --train-steps 250000 -d peppe -b 30 --init params.yml

Fitting an atomic dress from the training set.
 RMSE after substracting the dress: 1.112540e+00
INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': './', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': './', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [36]:

train_spec = tf.estimator.TrainSpec(input_fn=train, max_steps=1000)
eval_spec = tf.estimator.EvalSpec(input_fn=test)


In [37]:
tf.estimator.train_and_evaluate(model, train_spec, eval_spec)

INFO:tensorflow:Not using Distribute Coordinator.


INFO:tensorflow:Not using Distribute Coordinator.


INFO:tensorflow:Running training and evaluation locally (non-distributed).


INFO:tensorflow:Running training and evaluation locally (non-distributed).


INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.


...

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: ./model.ckpt-1000


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: ./model.ckpt-1000


INFO:tensorflow:Loss for final step: 3.3908906e-05.


INFO:tensorflow:Loss for final step: 3.3908906e-05.


({'METRICS/E_LOSS': 1.6163075,
  'METRICS/E_MAE': 1.084709,
  'METRICS/E_RMSE': 1.2713408,
  'loss': 1.6163075,
  'global_step': 1000},
 [])

In [ ]:

#generate a 80:20 test set. 
os.chdir(base_dir)
cwd = os.getcwd()
filelist = glob(cwd+'/PiNN/trainset.xyz')
dataset = lambda: load_qm9(filelist, split={'train':8, 'test':2})
train = lambda: dataset()['train'].repeat().shuffle(10).apply(sparse_batch(1))
test = lambda: dataset()['test'].repeat().apply(sparse_batch(1))

                    
#os.chdir(base_dir)
#!pwd
#os.chdir(base_dir+"/PiNN/")
#!pinn convert trainset.xyz -o 'train:8,test:2' 


# Train the poteinals 

In [ ]:
from ase import Atoms
datalist = [Atoms(elem) for elem in ['Cu', 'Ag', 'Au']]

datalist

In [ ]:
from pinn.io import list_loader

@list_loader()
def load_ase_list(atoms):
    import numpy as np
    datum = {'elems': atoms.numbers,
            'coord': atoms.positions,
            'e_data': 0.0}
    return datum

In [ ]:
import tensorflow as tf

dataset = load_ase_list(datalist)['train']
d = dataset.make_one_shot_iterator().get_next()
with tf.Session() as sess:
    print(sess.run(d))

In [ ]:
from pinn.io.base import list_loader

def _ase_spec(atoms):
    """Guess dataset format from an ASE atoms"""
    ds_spec = {
        'elems': {'dtype':  'int32', 'shape': [None]},
        'coord': {'dtype':  'float', 'shape': [None, 3]}}

    if atoms.pbc.any():
        ds_spec['cell'] = {'dtype': 'float', 'shape': [3, 3]}

    try:
        atoms.get_potential_energy()
        ds_spec['e_data'] = {'dtype': 'float', 'shape': []}
    except:
        pass

    try:
        atoms.get_forces()
        ds_spec['f_data'] = {'dtype': 'float', 'shape': [None, 3]}
    except:
        pass

    try:
        atoms.get_charges()
        ds_spec['q_data'] = {'dtype': 'float', 'shape': [None]}
    except:
        pass

    try:
        atoms.get_dipole_moment()
        ds_spec['d_data'] = {'dtype': 'float', 'shape': [3]}
    except:
        pass

    return ds_spec

def load_ase(dataset, splits=None, shuffle=True, seed=0):
    """
    Loads a ASE trajectory
    Args:
        dataset (str or ase.io.trajectory): a filename or trajectory
        splits (dict): key-val pairs specifying the ratio of subsets
        shuffle (bool): shuffle the dataset (only used when splitting)
        seed (int): random seed for shuffling
    """
    from ase.io import read

    if isinstance(dataset, str):
        dataset = read(dataset, index=':')

    ds_spec = _ase_spec(dataset[0])
    @list_loader(ds_spec=ds_spec)
    def _ase_loader(atoms):
        datum = {
            'elems': atoms.numbers,
            'coord': atoms.positions,
        }
        if 'cell' in ds_spec:
            datum['cell'] = atoms.cell[:]

        if 'e_data' in ds_spec:
            datum['e_data'] = atoms.get_potential_energy()

        if 'f_data' in ds_spec:
            datum['f_data'] = atoms.get_forces()

        if 'q_data' in ds_spec:
            datum['q_data'] = atoms.get_charges()

        if 'd_data' in ds_spec:
            datum['d_data'] = atoms.get_dipole_moment()
        return datum

    return _ase_loader(dataset, splits=splits, shuffle=shuffle, seed=seed)